In [33]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

lap_times_csv = "/content/drive/MyDrive/Colab Notebooks/archive/lap_times.csv"
results_csv = "/content/drive/MyDrive/Colab Notebooks/archive/results.csv"
races_csv = "/content/drive/MyDrive/Colab Notebooks/archive/races.csv"
qualifying_csv = "/content/drive/MyDrive/Colab Notebooks/archive/qualifying.csv"
status_csv = "/content/drive/MyDrive/Colab Notebooks/archive/status.csv"

lap_times_df = pd.read_csv(lap_times_csv)
results_df = pd.read_csv(results_csv)
races_df = pd.read_csv(races_csv)
qualifying_df = pd.read_csv(qualifying_csv)
status_df = pd.read_csv(status_csv)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
def prepare_grid_data(results_df):

  grid_df = results_df[['driverId', 'raceId', 'grid']].copy()
  return grid_df
display(prepare_grid_data(results_df))

,driverId,raceId,grid
0,1,18,1
1,2,18,5
2,3,18,7
3,4,18,11
4,5,18,3
...,...,...,...
26754,825,1144,14
26755,859,1144,12
26756,822,1144,9
26757,861,1144,20


In [35]:
def prepare_laps_led_data(lap_times_df, n=3):
  # Filter for laps where the driver was in the lead (position == 1)
  laps_led_df = lap_times_df[lap_times_df['position'] == 1].copy()

  # Count the number of laps led per driver per race
  laps_led_count = laps_led_df.groupby(['raceId', 'driverId']).size().reset_index(name='laps_led')

  # Sort by raceId to easily get the last n races and reset index for merging
  laps_led_count_sorted = laps_led_count.sort_values(by='raceId').reset_index(drop=True)

  # Calculate cumulative laps led over the last n races for each driver
  laps_led_last_n_races = laps_led_count_sorted.groupby('driverId')['laps_led'].rolling(window=n, min_periods=1).sum().reset_index()
  laps_led_last_n_races = laps_led_last_n_races.rename(columns={'laps_led': f'laps_led_last_{n}_races'})

  # Merge with raceId to get the raceId for each cumulative sum
  # Merge on driverId and the sequential index generated by reset_index()
  laps_led_last_n_races = laps_led_last_n_races.merge(laps_led_count_sorted[['raceId', 'driverId']].reset_index(), left_on=['driverId', 'level_1'], right_on=['driverId', 'index'])
  laps_led_last_n_races = laps_led_last_n_races[['driverId', 'raceId', f'laps_led_last_{n}_races']]

  return laps_led_last_n_races
display(prepare_laps_led_data(lap_times_df, n=3))

,driverId,raceId,laps_led_last_3_races
0,1,10,58.0
1,1,11,89.0
2,1,13,109.0
3,1,14,108.0
4,1,17,93.0
...,...,...,...
1375,857,1133,47.0
1376,857,1134,50.0
1377,857,1135,53.0
1378,857,1136,40.0


In [36]:
def prepare_best_lap_data(lap_times_df, races_df):
  # Merge lap times with races to get circuitId for each lap
  lap_times_circuits = lap_times_df.merge(races_df[['raceId', 'circuitId']], on='raceId')

  # Ensure milliseconds column is numeric, coercing errors to NaN
  lap_times_circuits['milliseconds'] = pd.to_numeric(lap_times_circuits['milliseconds'], errors='coerce')

  # Find the minimum milliseconds for each driver at each circuit
  best_laps = lap_times_circuits.groupby(['driverId', 'circuitId'])['milliseconds'].min().reset_index(name='best_lap_milliseconds')

  return best_laps
display(prepare_best_lap_data(lap_times_df, races_df))

,driverId,circuitId,best_lap_milliseconds
0,1,1,80613
1,1,2,94452
2,1,3,92798
3,1,4,76676
4,1,5,86529
...,...,...,...
2818,861,32,80090
2819,861,69,97611
2820,861,73,107274
2821,861,80,96867


In [37]:
def prepare_dnf_rate_data(results_df, status_df):
  # Merge results with status to get the status of each result
  results_with_status = results_df.merge(status_df[['statusId', 'status']], on='statusId')

  # Determine which statuses indicate a DNF
  dnf_statuses = status_df[~status_df['status'].str.contains('Finished', na=False)]['statusId'].tolist()

  # Add a column indicating if a result is a DNF
  results_with_status['is_dnf'] = results_with_status['statusId'].isin(dnf_statuses)

  # Merge with races to get the year of each race
  results_with_year = results_with_status.merge(races_df[['raceId', 'year']], on='raceId')

  # Calculate the total number of races and the number of DNFs for each driver in each season
  dnf_counts = results_with_year.groupby(['driverId', 'year']).agg(
      total_races=('raceId', 'count'),
      dnf_count=('is_dnf', 'sum')
  ).reset_index()

  # Calculate the DNF rate
  dnf_counts['dnf_rate_season'] = dnf_counts['dnf_count'] / dnf_counts['total_races']

  # Select and return the relevant columns
  dnf_rate_season = dnf_counts[['driverId', 'year', 'dnf_rate_season']]

  return dnf_rate_season
display(prepare_dnf_rate_data(results_df, status_df))

,driverId,year,dnf_rate_season
0,1,2007,0.176471
1,1,2008,0.111111
2,1,2009,0.470588
3,1,2010,0.210526
4,1,2011,0.157895
...,...,...,...
3206,859,2023,0.400000
3207,859,2024,0.333333
3208,860,2024,0.000000
3209,861,2024,0.555556


In [38]:
def prepare_constructor_win_data(results_df, races_df):
  # Merge results with races to get circuitId for each race
  results_circuits = results_df.merge(races_df[['raceId', 'circuitId']], on='raceId')

  # Filter for races where the constructor's driver finished in 1st position
  constructor_wins = results_circuits[results_circuits['positionOrder'] == 1].copy()

  # Count the number of wins for each constructor at each circuit
  constructor_wins_count = constructor_wins.groupby(['constructorId', 'circuitId']).size().reset_index(name='constructor_wins_at_circuit')

  return constructor_wins_count
display(prepare_constructor_win_data(results_df, races_df))

,constructorId,circuitId,constructor_wins_at_circuit
0,1,1,6
1,1,2,2
2,1,4,4
3,1,5,2
4,1,6,15
...,...,...,...
427,208,1,1
428,208,24,1
429,211,3,1
430,213,14,1


In [39]:
def prepare_driver_win_data(results_df, races_df):
  # Merge results with races to get circuitId for each race
  results_circuits = results_df.merge(races_df[['raceId', 'circuitId']], on='raceId')

  # Filter for races where the driver finished in 1st position
  driver_wins = results_circuits[results_circuits['positionOrder'] == 1].copy()

  # Count the number of wins for each driver at each circuit
  driver_wins_count = driver_wins.groupby(['driverId', 'circuitId']).size().reset_index(name='driver_wins_at_circuit')

  return driver_wins_count
display(prepare_driver_win_data(results_df, races_df))

,driverId,circuitId,driver_wins_at_circuit
0,1,1,2
1,1,2,1
2,1,3,5
3,1,4,6
4,1,5,2
...,...,...,...
682,847,18,1
683,847,70,1
684,847,80,1
685,857,11,1


In [40]:
grid_df = prepare_grid_data(results_df)
laps_led_df = prepare_laps_led_data(lap_times_df)
best_lap_df = prepare_best_lap_data(lap_times_df, races_df)
driver_wins_df = prepare_driver_win_data(results_df, races_df)
constructor_wins_df = prepare_constructor_win_data(results_df, races_df)
dnf_rate_df = prepare_dnf_rate_data(results_df, status_df)

In [41]:
merged_df = races_df[['raceId', 'circuitId', 'year']].merge(results_df[['raceId', 'driverId', 'constructorId', 'grid', 'positionOrder']], on='raceId', how='inner')

merged_df = merged_df.merge(laps_led_df, on=['driverId', 'raceId'], how='left')
merged_df = merged_df.merge(best_lap_df, on=['driverId', 'circuitId'], how='left')
merged_df = merged_df.merge(driver_wins_df, on=['driverId', 'circuitId'], how='left')
merged_df = merged_df.merge(constructor_wins_df, on=['constructorId', 'circuitId'], how='left')
merged_df = merged_df.merge(dnf_rate_df, on=['driverId', 'year'], how='left')

display(merged_df)

,raceId,circuitId,year,driverId,constructorId,grid,positionOrder,laps_led_last_3_races,best_lap_milliseconds,driver_wins_at_circuit,constructor_wins_at_circuit,dnf_rate_season
0,1,1,2009,18,23,1,1,58.0,85982.0,3.0,1.0,0.058824
1,1,1,2009,22,23,2,2,NaN,84179.0,NaN,1.0,0.058824
2,1,1,2009,15,7,20,3,NaN,86275.0,NaN,NaN,0.352941
3,1,1,2009,10,7,19,4,NaN,88416.0,NaN,NaN,0.200000
4,1,1,2009,4,4,10,5,NaN,80476.0,1.0,2.0,0.294118
...,...,...,...,...,...,...,...,...,...,...,...,...
26754,1144,24,2024,825,210,14,16,NaN,85637.0,NaN,NaN,0.454545
26755,1144,24,2024,859,215,12,17,NaN,88751.0,NaN,NaN,0.333333
26756,1144,24,2024,822,15,9,18,NaN,86862.0,1.0,NaN,0.791667
26757,1144,24,2024,861,3,20,19,NaN,89411.0,NaN,NaN,0.555556


In [42]:
print(merged_df.isnull().sum())

raceId                             0
circuitId                          0
year                               0
driverId                           0
constructorId                      0
grid                               0
positionOrder                      0
laps_led_last_3_races          25379
best_lap_milliseconds          14419
driver_wins_at_circuit         21285
constructor_wins_at_circuit    15352
dnf_rate_season                    0
dtype: int64
